
# Schnittstellen und Vererbung in Python

## Lernziele
Nach diesem Notebook kannst du …
- erklären, was *Schnittstellen* in Python bedeuten (ABC & Protocol) und warum man sie nutzt,
- Vererbung mit Basisklassen und Unterklassen anwenden,
- Methoden überschreiben und `super()` sinnvoll einsetzen,
- Polymorphie mit gemeinsamen Schnittstellen nutzen,
- den Unterschied zwischen Vererbung (*ist-ein*) und Komposition (*hat-ein*) beschreiben.



## 1. Warm-up: Ein einfaches `Auto`
Wir starten ohne Interfaces/ABCs und erinnern uns an grundlegende OOP-Begriffe.


In [ ]:

class Auto:
    def __init__(self, marke: str, modell: str, maximalgeschwindigkeit: int):
        self.marke = marke
        self.modell = modell
        self.max_v = maximalgeschwindigkeit
        self.geschwindigkeit = 0
        self.motor_an = False

    def starte_motor(self):
        self.motor_an = True
        print(f"{self.marke} {self.modell}: Motor gestartet.")

    def stoppe_motor(self):
        self.motor_an = False
        self.geschwindigkeit = 0
        print(f"{self.marke} {self.modell}: Motor gestoppt.")

    def beschleunige(self, delta: int):
        if not self.motor_an:
            print("Bitte erst den Motor starten!")
            return
        self.geschwindigkeit = min(self.geschwindigkeit + delta, self.max_v)
        print(f"Tempo: {self.geschwindigkeit} km/h")

# Demo
a = Auto("VW", "Golf", 200)
# starte den Motor
# beschleunige auf 50
# stoppe den Motor



## 2. Schnittstellen-Idee mit `Protocol` (Duck Typing)
In Python kann man mit `typing.Protocol` eine **strukturierte Schnittstelle** beschreiben, die Klassen *implizit* erfüllen können.


In [ ]:

from typing import Protocol

class Fahrbar(Protocol):
    def starte_motor(self) -> None: ...
    def stoppe_motor(self) -> None: ...
    def beschleunige(self, delta: int) -> None: ...

# Unsere Klasse Auto erfüllt diese Schnittstelle bereits – ganz ohne `implements`-Schlüsselwort.
def fahre_probe(fahrzeug: Fahrbar):
    fahrzeug.starte_motor()
    fahrzeug.beschleunige(30)
    fahrzeug.stoppe_motor()

# Eine völlig andere Klasse kann die Schnittstelle ebenfalls erfüllen
class Fahrrad:
    def __init__(self):
        self.tempo = 0
    def starte_motor(self):
        print("Fahrrad: hat keinen Motor – wir tun so, als wäre es startbereit 🚲")
    def stoppe_motor(self):
        self.tempo = 0
        print("Fahrrad: Stopp.")
    def beschleunige(self, delta: int):
        self.tempo += delta
        print(f"Fahrrad-Tempo: {self.tempo} km/h")

# Polymorphie über die gemeinsame Schnittstelle
print("-- Probe mit Auto --")
fahre_probe(Auto("BMW","3er",240))
print("-- Probe mit Fahrrad --")
fahre_probe(Fahrrad())


### ⚠️ Gegenbeispiel: Eine Klasse erfüllt das `Protocol` nicht vollständig

Bei einem `Protocol` wird **nicht erzwungen**, dass eine Klasse wirklich alle Methoden implementiert.  
Python prüft das **nicht zur Laufzeit**, sondern optional nur durch **Typprüfer** (z. B. VSCode mit Typprüfung).

Das bedeutet:
- Wenn eine Klasse **eine oder mehrere Methoden des Protocols fehlt**,  
  funktioniert der Code zunächst trotzdem — bis die fehlende Methode **aufgerufen** wird.
- Dann entsteht ein **`AttributeError`** zur Laufzeit.

In [ ]:
from typing import Protocol

class Fahrbar(Protocol):
    def starte_motor(self) -> None: ...
    def stoppe_motor(self) -> None: ...
    def beschleunige(self, delta: int) -> None: ...

# Diese Klasse erfüllt das Protocol NICHT vollständig (es fehlt stoppe_motor)
class Skateboard:
    def starte_motor(self):
        print("Skateboard: kein Motor vorhanden, aber wir tun so 😉")
    def beschleunige(self, delta: int):
        print(f"Skateboard rollt mit +{delta} km/h")

def fahre_probe(fahrzeug: Fahrbar):
    fahrzeug.starte_motor()
    fahrzeug.beschleunige(20)
    fahrzeug.stoppe_motor()   # <-- diese Methode fehlt beim Skateboard!

# Test
s = Skateboard()
try:
    fahre_probe(s)
except AttributeError as e:
    print("❗ Laufzeitfehler:")
    print(e)

## Exkurs: Typprüfung über VS-Code
1.	Gehe zu File → Preferences → Settings (oder über Ctrl + ,)
2.	Suche nach: “python type checking”
3.	Stelle Python › Analysis: Type Checking Mode ein auf:

-	off – ausgeschaltet
-	basic – grundlegende Typprüfung
-	strict – sehr strenge Typprüfung (empfohlen für Projekte)


## 3. Schnittstellen mit `ABC` (Abstract Base Class)
Alternativ kann man *abstrakte Basisklassen* definieren. Diese können **abstrakte Methoden** erzwingen.  
Eine Klasse, die davon erbt, **muss** die abstrakten Methoden implementieren.


In [ ]:

from abc import ABC, abstractmethod

class Fahrzeug(ABC):
    @abstractmethod
    def starte_motor(self) -> None: ...
    @abstractmethod
    def stoppe_motor(self) -> None: ...
    @abstractmethod
    def beschleunige(self, delta: int) -> None: ...

class VerbrennerAuto(Fahrzeug):
    def __init__(self, marke: str, modell: str):
        self.marke = marke
        self.modell = modell
        self.motor_an = False
        self.geschwindigkeit = 0

    def starte_motor(self) -> None:
        self.motor_an = True
        print(f"{self.marke} {self.modell} (Benzin/Diesel): Motor an.")

    def stoppe_motor(self) -> None:
        self.motor_an = False
        self.geschwindigkeit = 0
        print(f"{self.marke} {self.modell} (Benzin/Diesel): Motor aus.")

    def beschleunige(self, delta: int) -> None:
        if not self.motor_an:
            print("Motor ist aus.")
            return
        self.geschwindigkeit += delta
        print(f"{self.marke} {self.modell}: {self.geschwindigkeit} km/h")

# Demo
v = VerbrennerAuto("Audi", "A4")
v.starte_motor()
v.beschleunige(40)
v.stoppe_motor()


### ❌ Gegenbeispiel: Unvollständige Implementierung einer abstrakten Basisklasse

Wenn eine Klasse von einer abstrakten Basisklasse (`ABC`) **erbt**, aber **nicht alle abstrakten Methoden implementiert**, darf **keine Instanz** dieser Klasse erzeugt werden.  

Python schützt uns damit vor unvollständigen Implementierungen – die Klasse gilt als *abstrakt*.

Wir sehen das im folgenden Beispiel: `DefektesAuto` erbt von `Fahrzeug`, implementiert aber **nicht** alle Methoden.

In [ ]:
class DefektesAuto(Fahrzeug):
    def starte_motor(self):
        print("Motor gestartet – aber sonst fehlt was...")

# Wir haben `stoppe_motor` und `beschleunige` NICHT implementiert!

try:
    d = DefektesAuto()
except TypeError as e:
    print("❗Fehler beim Erzeugen der Instanz:")
    print(e)

Erklärung:
- `TypeError` zeigt an, dass noch abstrakte Methoden (stoppe_motor, beschleunige) fehlen.
- Erst wenn alle abstrakten Methoden überschrieben wurden, darf die Klasse instanziiert werden.


#### 🔍 Vergleich
| Aspekt | `Protocol` | `ABC` (Abstract Base Class) |
|--------|-------------|------------------------------|
| Beziehung | über gleiche Methoden-Signatur | über Vererbung |
| Verbindung zur Klasse | keine erforderlich | Klasse **muss** erben |
| Prüfung | durch Typprüfung | zur Laufzeit (Instanziierung verboten, wenn unvollständig) |
| Flexibilität | sehr hoch | strenger |
| Zweck | lose definierte Schnittstelle | gemeinsame Oberklasse erzwingen |




## 4. Vererbung & Überschreiben: `ElektroAuto` als Unterklasse
Unterklassen können **Methoden überschreiben** und mit `super()` die Basisklasse einbeziehen.


In [ ]:

class ElektroAuto(VerbrennerAuto):
    def __init__(self, marke: str, modell: str, akku_kapazitaet_kwh: float):
        super().__init__(marke, modell)
        self.akku_kapazitaet_kwh = akku_kapazitaet_kwh
        self.ladestand = 100.0  # Prozent

    # Überschreibt die Start-Logik
    def starte_motor(self) -> None:
        self.motor_an = True
        print(f"{self.marke} {self.modell} (EV): System hochgefahren – lautlos ✅")

    def lade(self, prozent: float) -> None:
        self.ladestand = min(100.0, max(0.0, self.ladestand + prozent))
        print(f"Ladestand: {self.ladestand:.1f}%")

    # Überschreibt Beschleunigung (EVs sind oft spritziger 😉)
    def beschleunige(self, delta: int) -> None:
        if not self.motor_an:
            print("System ist aus.")
            return
        # kleine Sonderlogik für EV
        self.geschwindigkeit += int(delta * 1.2)
        print(f"{self.marke} {self.modell}: {self.geschwindigkeit} km/h (EV-Boost)")
        self.lade(-0.1*delta)

# Demo
e = ElektroAuto("Tesla", "Model 3", 60)
e.starte_motor()
e.beschleunige(50)
e.stoppe_motor()



## 5. Polymorphie in Aktion
Eine Funktion kann *jedes* Objekt annehmen, das die gemeinsame Schnittstelle (`Fahrzeug`) erfüllt.


In [ ]:

def starte_und_fahre(fzg: Fahrzeug) -> None:
    fzg.starte_motor()
    fzg.beschleunige(20)
    fzg.stoppe_motor()

print("-- Polymorph mit VerbrennerAuto --")
starte_und_fahre(VerbrennerAuto("Opel","Corsa"))
print("-- Polymorph mit ElektroAuto --")
starte_und_fahre(ElektroAuto("Renault","Megane E-Tech", 60))



## 6. Komposition (*hat-ein*) vs. Vererbung (*ist-ein*)
Nicht alles sollte geerbt werden. Oft ist **Komposition** klarer und flexibler.


In [ ]:

class Motor:
    def __init__(self, typ: str):
        self.typ = typ
        self.laufend = False

    def starten(self):
        self.laufend = True
    def stoppen(self):
        self.laufend = False

class KompositionsAuto:
    def __init__(self, marke: str, modell: str, motor: Motor):
        self.marke = marke
        self.modell = modell
        self.motor = motor
        self.tempo = 0

    def starte_motor(self):
        self.motor.starten()
        print(f"{self.marke} {self.modell}: {self.motor.typ}-Motor gestartet.")

    def beschleunige(self, delta: int):
        if not self.motor.laufend:
            print("Motor ist aus.")
            return
        self.tempo += delta
        print(f"{self.marke} {self.modell}: {self.tempo} km/h")

# Demo
k = KompositionsAuto("Toyota","Yaris", Motor("Hybrid"))
k.starte_motor()
k.beschleunige(30)


**Aufgabe**

Erkläre den Unterschied zwischen Vererbung und Komposition. Beschreibe, wie die Variante mit Vererbung aussähe.


## <font color=red >Übung</font>

1. **Interface-Übung (Protocol):** Definiere ein `Lenkbar`-Protocol mit `lenke(winkel: int)`.  
   Implementiere `lenke` in `Auto` und `Fahrrad`, schreibe eine Funktion `slalom(obj)`, die `lenke` 3× aufruft.

2. **ABC-Übung:** Erzeuge eine abstrakte Klasse `Fluggeraet` mit `abheben()` und `landen()`.  
   Implementiere `Drohne(Fluggeraet)` und `Flugzeug(Fluggeraet)`.

3. **Vererbung & super():** Erzeuge `Rennwagen(VerbrennerAuto)`, der beim `beschleunige` ein Limit von `+100` pro Schritt setzt.  
   Nutze `super().beschleunige(...)` und ergänze eine spezielle Ausgabe.

4. **Komposition:** Baue eine Klasse `Akku` (mit `kapazitaet_kwh`, `stand_prozent`) und verwende sie in `ElektroAuto` anstelle des float-Feldes.  
   Schreibe eine Methode `lade_auf(kwh: float)` die den Prozentstand passend erhöht (vereinfacht).

> **Hinweis:** Teste deine Lösungen mit `assert`-Statements und kurzen Demos.









+---------------------------------------------------+
| t1 : Tablet                                       |
+---------------------------------------------------+
| ladezustandInProzent = 80                         |
| installierteApps = ["YouTube", "Mail"]            |
+---------------------------------------------------+



+---------------------------------------------------+
| b : Buch                                          |
+---------------------------------------------------+
| titel = "Python 101"                              |
| autor = "Müller"                                  |
| verlag = "Tech-Verlag"                            |
| fach = "Informatik"                               |
| preis = 29.99                                     |
+---------------------------------------------------+
| ausleihen()                                       |
+---------------------------------------------------+


+---------------------------------------------------+
| h : Handy                                         |
+---------------------------------------------------+
| name = "Samsung S20"                              |
| ladezustand = 20                                  |
+---------------------------------------------------+
| aufladen()                                        |
+---------------------------------------------------+


+---------------------------------------------------+
| t2 : Tablet2                                      |
+---------------------------------------------------+
| name = "iPad"                                     |
| ladezustand = 50                                  |
+---------------------------------------------------+
| ausleihen()                                       |
| aufladen()                                        |
+---------------------------------------------------+



                 <<interface>>

                    Lenkbar

                -----------------

                +lenke(winkel:GZ)
 
 
        . . . . . . . . . . . . . . . . . .

        .                                 .

        v                                 v
 
+-----------------------+        +----------------+

|         Auto          |        |    Fahrrad     |

+-----------------------+        +----------------+

| marke: Text           |        |                |

| modell: Text          |        |                |

+-----------------------+        +----------------+

| +lenke(winkel:GZ)     |        | +lenke(winkel:GZ) |

+-----------------------+        +----------------+

 

